In [1]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [2]:
import hashlib

def generate_document_id(doc):
    # combined = f"{doc['course']}-{doc['question']}"
    combined = f"{doc['course']}-{doc['question']}-{doc['text'][:10]}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]
    return document_id

In [3]:
for doc in documents:
    doc['id'] = generate_document_id(doc)

In [5]:
documents[42]

{'text': 'Error: Makefile:2: *** missing separator.  Stop.\nSolution: Tabs in document should be converted to Tab instead of spaces. Follow this stack.',
 'section': 'General course-related questions',
 'question': 'VS Code: Tab using spaces',
 'course': 'data-engineering-zoomcamp',
 'id': 'eb9d376f'}

In [6]:
from collections import defaultdict

In [8]:
hashes = defaultdict(list)

for doc in documents:
    doc_id = doc['id']
    hashes[doc_id].append(doc)

In [9]:
len(hashes), len(documents)

(947, 948)

In [10]:
for k, values in hashes.items():
    if len(values) > 1:
        print(k, len(values))

593f7569 2


In [11]:
hashes['593f7569']

[{'text': "They both do the same, it's just less typing from the script.\nAsked by Andrew Katoch, Added by Edidiong Esu",
  'section': '6. Decision Trees and Ensemble Learning',
  'question': 'Does it matter if we let the Python file create the server or if we run gunicorn directly?',
  'course': 'machine-learning-zoomcamp',
  'id': '593f7569'},
 {'text': "They both do the same, it's just less typing from the script.",
  'section': '6. Decision Trees and Ensemble Learning',
  'question': 'Does it matter if we let the Python file create the server or if we run gunicorn directly?',
  'course': 'machine-learning-zoomcamp',
  'id': '593f7569'}]

In [12]:
import json

In [13]:
with open('documents-with-ids.json', 'wt') as f_out:
    json.dump(documents, f_out, indent=2)

In [14]:
!head documents-with-ids.json

[
  {
    "text": "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  \u201cOffice Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon\u2019t forget to register in DataTalks.Club's Slack and join the channel.",
    "section": "General course-related questions",
    "question": "Course - When will the course start?",
    "course": "data-engineering-zoomcamp",
    "id": "c02e79ef"
  },
  {
    "text": "GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites",


In [15]:
prompt_template = """
You emulate a student who's taking our course.
Formulate 5 questions this student might ask based on a FAQ record. The record
should contain the answer to the questions, and the questions should be complete and not too short.
If possible, use as fewer words as possible from the record. 

The record:

section: {section}
question: {question}
answer: {text}

Provide the output in parsable JSON without using code blocks:

["question1", "question2", ..., "question5"]
""".strip()

In [16]:
from openai import OpenAI
client = OpenAI()

In [17]:
def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )

    json_response = response.choices[0].message.content
    return json_response

In [18]:
from tqdm.auto import tqdm

In [19]:
results = {}

In [20]:
# aroud 4$ for almost 1000 requests
for doc in tqdm(documents): 
    doc_id = doc['id']
    if doc_id in results:
        continue

    questions = generate_questions(doc)
    results[doc_id] = questions

  0%|          | 0/948 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [24]:
import pickle

In [25]:
with open('results.bin', 'rb') as f_in:
    results = pickle.load(f_in)

In [26]:
results['1f6520ca']

'["Where can I find the prerequisites for this course?", "How do I check the prerequisites for this course?", "Where are the course prerequisites listed?", "What are the requirements for joining this course?", "Where is the list of prerequisites for the course?"]'

In [28]:
parsed_resulst = {}

for doc_id, json_questions in results.items():
    try:
        parsed_resulst[doc_id] = json.loads(json_questions)
    except json.JSONDecodeError:
        print(doc_id)
        continue

58c9f99f


In [29]:
doc_index = {d['id']: d for d in documents}

In [30]:
final_results = []

for doc_id, questions in parsed_resulst.items():
    course = doc_index[doc_id]['course']
    for q in questions:
        final_results.append((q, course, doc_id))

In [31]:
import pandas as pd

In [32]:
df = pd.DataFrame(final_results, columns=['question', 'course', 'document'])

In [33]:
df.to_csv('my-ground-truth-data.csv', index=False)

In [78]:
!head ground-truth-data.csv

question,course,document
When does the course begin?,data-engineering-zoomcamp,c02e79ef
How can I get the course schedule?,data-engineering-zoomcamp,c02e79ef
What is the link for course registration?,data-engineering-zoomcamp,c02e79ef
How can I receive course announcements?,data-engineering-zoomcamp,c02e79ef
Where do I join the Slack channel?,data-engineering-zoomcamp,c02e79ef
Where can I find the prerequisites for this course?,data-engineering-zoomcamp,1f6520ca
How do I check the prerequisites for this course?,data-engineering-zoomcamp,1f6520ca
Where are the course prerequisites listed?,data-engineering-zoomcamp,1f6520ca
What are the requirements for joining this course?,data-engineering-zoomcamp,1f6520ca


In [37]:
df.sample(10)

,question,course,document
3679,How should I edit the quick_install.sh file to...,machine-learning-zoomcamp,5cb58698
3316,How does flow_from_directory identify class na...,machine-learning-zoomcamp,f20a3479
2621,Which scalers should be used for numeric and c...,machine-learning-zoomcamp,34a8edb0
57,Why am I unable to get a certificate for the s...,data-engineering-zoomcamp,cb257ee5
2745,Where can I find more information about gettin...,machine-learning-zoomcamp,2ab49e43
2853,What arguments does the roc_auc_score function...,machine-learning-zoomcamp,147577f5
3420,What causes ERROR [internal] load metadata for...,machine-learning-zoomcamp,0edeb016
1224,What should I do if I get an 'Access Denied' e...,data-engineering-zoomcamp,14a876ea
921,What causes an error when trying to delete a b...,data-engineering-zoomcamp,6290a1a6
1510,How can I create an external table to avoid pa...,data-engineering-zoomcamp,2aadd232
